### Modeling with MLOps

In [1]:
# Importing Pandas an Numpy Libraries to use on manipulating our Data
import pandas as pd
import numpy as np
import seaborn as sns
import dvc.api
from sklearn import preprocessing

import sys
sys.path.append("../scripts")

from modeling_pipeline import (
    run_train_pipeline, 
    get_pipeline,
    dvc_get_data)
from clean_data import DataCleaner
import os
import mlflow

ModuleNotFoundError: No module named 'modeling_pipeline'

In [26]:
cleaner= DataCleaner()
data_name="dset.csv"

In [14]:
df = pd.read_csv('../data/AdSmartABdata.csv')
cleaned_df=cleaner.drop_unresponsive(df)
control_df=cleaned_df[cleaned_df['experiment']=='control']
exposed_df=cleaned_df[cleaned_df['experiment']=='exposed']

combined_df=cleaned_df.drop(columns=['no'])
combined_df.rename(columns={'yes':'response'},inplace=True)
combined_df

,auction_id,experiment,date,hour,device_make,platform_os,browser,response
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0
16,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1
20,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,6,Chrome Mobile,0
23,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1
27,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0
...,...,...,...,...,...,...,...,...
8059,ffa08ff9-a132-4051-aef5-01a9c79367bc,exposed,2020-07-05,21,Generic Smartphone,6,Chrome Mobile,1
8063,ffb176df-ecd2-45d3-b05f-05b173a093a7,exposed,2020-07-04,1,Generic Smartphone,6,Chrome Mobile,1
8064,ffb79718-6f25-4896-b6b3-e58b80a6e147,control,2020-07-09,7,Generic Smartphone,6,Chrome Mobile,0
8069,ffca1153-c182-4f32-9e90-2a6008417497,control,2020-07-10,16,Generic Smartphone,6,Chrome Mobile,0


### split data to browser and platform_os

In [18]:
def feature_data(df):
    browser_df=df.loc[:,df.columns!='platform_os']
    platform_df=df.loc[:,df.columns!='browser']

    return browser_df,platform_df

In [24]:
# def save_encoded_df(encoded_df):
#     browser_df,platform_df=feature_data(encoded_df)
#     browser_df.to_csv('../data/browser.csv',index=False)
#     platform_df.to_csv('../data/platform.csv',index=False)

Encoding categorical variables(label encoding)

In [22]:
def encode_labels(combined_df):
    date_encoder = preprocessing.LabelEncoder()
    device_encoder = preprocessing.LabelEncoder()
    browser_encoder = preprocessing.LabelEncoder()
    experiment_encoder = preprocessing.LabelEncoder()
    response_encoder = preprocessing.LabelEncoder()
    
    combined_df['date'] = date_encoder.fit_transform(combined_df['date'])
    combined_df['device_make'] = device_encoder.fit_transform(combined_df['device_make'])
    combined_df['browser'] = browser_encoder.fit_transform(combined_df['browser'])
    combined_df['experiment'] = experiment_encoder.fit_transform(combined_df['experiment'])
    combined_df['browser'] = response_encoder.fit_transform(combined_df['browser'])
    combined_df['response'] = response_encoder.fit_transform(combined_df['response'])


    
    return combined_df

In [25]:
# encoded_df = encode_labels(combined_df)
# save_encoded_df(encoded_df)

In [ ]:
encoded_df = encode_labels(combined_df)
browser_df,platform_df=feature_data(encoded_df)

In [ ]:
browser_df.to_csv(f'../data/{data_name}',index=False)


In [ ]:
platform_df.to_csv(f'../data/{data_name}',index=False)


In [ ]:
# versioning browser 
path = f'../data/{data_name}'
repo = '/temp/dvc-storage/'
rev = 'browser-v1'
data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
collected_data = pd.read_csv(data_url)

In [ ]:
# versioning browser 
path = f'../data/{data_name}'
repo = '/temp/dvc-storage/'
rev = 'platform-v1'
data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
collected_data = pd.read_csv(data_url)